# Coursera Capstone 
# Week 4 Assignment - The Battle of neighborhoods(Week 1)
## Introduction

### Question 1 :- 
Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly describe your audience and why they would care about your problem.

## Intro :-
I am a Software developer working in Maryland. I have to move to Miami,Florida. I used to live in Owings Mills, Maryland with walking distance to the Grand Junction Metro Station with great connectivity to Baltimore Downtown as well as Washington, D.C., all shopping stores, groceries and pharmacy was nearby to my apartment. But since I am moving to Florida I want to use the skills learned during IBM Data Science Specialization course to help me find a suitable apartment that meets the price range along with good locality which should be connected easily to public transport such as metros and buses and have groceries stores and good restaurants nearby. I can use apartment and renting websites to help me with my situation but I want to apply my learning and utilize the tools I used to help solve the problem. To help solve the problem, I will evaluate and compare the apartments available for renting in Miami, Florida and they should meet the following measures:

- Apartment must be 2 bedrooms.
- Desired location is near a metro station in the Manhattan area and within (1.5 km) radius or less than 30 minutes walking distance.
- Price of rent not exceed $2,000 per month.
- Amenities in the selected neighborhood shall be similar to my current residence.
- Desirable to have venues such as coffee shops,good Restaurants,Gym and Grocery stores nearby.
- As a reference, I have included a map of venues near current residence in Maryland.

### Business Problem

The challenge is to find a suitable apartment for rent in Miami, Florida that meets with the demands of appropriate location, price and venues. The data required to resolve this challenge is described in section 2.

### Project Purpose :-

This information will be helpful for someone moving to a new city and will help them to find a good apartment, as the basic questions for renting a reasonable apartment in a city are being answered. It will also help individuals interested in exploring coffee shops and restaurants in any city. Lastly, it is serving the purpose of helping me improve my Data Science skills as I can apply in this project.

## Question 2 :-

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. You can absolutely use other datasets in combination with the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

### About Data Requirements

- List of neighborhoods of Miami, Florida with their geodata (latitude and longitude).
- List of apartments for rent in Miami area with their addresses and price.
- List of Subway metro stations in Miami with their relative location.
- Preferably, a list of apartments for rent with additional information, such as price, address, area, # of beds
- Venues for each Miami neighborhoods ( that can be clustered).
- Venues for subway metro stations.

### Use of data

- Use Foursquare and geopy data to map top 10 venues for all Miami neighborhoods and clustered in groups ( as per Course LAB).
- Use foursquare and geopy data to map the location of metro stations , separately and on top of the above clustered map in order to be able to identify the venues and amenities near each metro station, or explore each public transport location separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the public transport locations.
- Create a map that depicts, for instance, the average rental price per square ft, around a radious of (1.5 km) around each subway station - or a similar metric. I will be able to quickly point to the popups to know the relative price per subway area.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.
- Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metrorail, etc.

### Decision of renting and answer the below questions :-

- what is the cost of rent (per square ft) around a mile radius from each Metrorail station?
- what is the area of Miami with best rental pricing that meets criteria established?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Miami neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

## Reference of venues around current residence in Maryland for comparison to Miami.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geocoder
!pip install geopy
import geocoder
import os
!pip install folium
import folium # map rendering library

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

print('Libraries imported.')

     |████████████████████████████████| 98 kB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 5.9 MB/s  eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    vincent-0.4.4

In [6]:
# Owings Mills, Maryland 21112 United States of America
address = 'Owings Mills, Maryland'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Maryland residence are {}, {}.'.format(latitude, longitude))

ConfigurationError: Using Nominatim with default or sample `user_agent` "geopy/2.1.0" is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`.

In [3]:
neighborhood_latitude= 39.4010753
neighborhood_longitude = -76.796299

In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
CLIENT_ID = 'XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215' # My Foursquare ID
CLIENT_SECRET = 'LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB' # My Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215
CLIENT_SECRET: LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB


In [5]:
url= 'https://api.foursquare.com/v2/venues/explore?&client_id=XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215&client_secret=LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB&v=20180604&ll=39.4010753,-76.796299&radius=500&limit=100'
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XA4L4ESICRTSGS5GINHBFROT2VMUXSDM0OIM3WBGZFBJ5215&client_secret=LDGFKKQCGQXK5D4E4BOEYLDBL54ZOXCYAYCYWZBQQL0BY5CB&v=20180604&ll=39.4010753,-76.796299&radius=500&limit=100'

In [7]:
results = requests.get(url).json()
#results

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
SBnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SBnearby_venues =SBnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SBnearby_venues['venue.categories'] = SBnearby_venues.apply(get_category_type, axis=1)

# clean columns
SBnearby_venues.columns = [col.split(".")[-1] for col in SBnearby_venues.columns]

SBnearby_venues.head(10)

<ipython-input-9-36d8873e2a78>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  SBnearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Asian Kitchen,Asian Restaurant,39.399578,-76.799104
1,Taj Palace Owings MIlls,North Indian Restaurant,39.399413,-76.800307
2,Artful Gourmet Bistro,New American Restaurant,39.399300,-76.801364
3,Red Robin Gourmet Burgers and Brews,Burger Joint,39.403242,-76.794221
4,Yuki Sushi,Sushi Restaurant,39.399449,-76.799871
5,Baskin-Robbins,Ice Cream Shop,39.399234,-76.799813
6,Olive Garden,Italian Restaurant,39.404531,-76.793403
7,"Brookside's Pizzeria, The Original NY Pizza Co.",Pizza Place,39.399343,-76.800756
8,The Greene Turtle,Sports Bar,39.402854,-76.794645
9,Red Lobster,Seafood Restaurant,39.404619,-76.792716


In [10]:

# create map of Southbank Melbourne using latitude and longitude values
map_sb = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SBnearby_venues['lat'], SBnearby_venues['lng'], SBnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sb)  
    
map_sb

NameError: name 'latitude' is not defined